# Tutoriel pour **InfluxDB**

Bienvenue sur ce tutoriel ! Vous allez découvrir les principales fonctionnalitées de InfluxDB, une base de donnée non relationnelle orientée séries temporelles.
Elle peut être utilisée dans de nombreux domaines, en stockant par exemple :
    - les données des capteurs industriels,
    - les métriques de performance des serveurs,
    - les mesures des précipitations,
    - le cours en bourse.

## ❓ Comment se présente la base de donnée ?

L'équivalent d'une occurance est un *point* qui est composé d'un *timestamp*, *measurement*, *tag keys*, *tag values* et *field key*.
Un groupe de points avec les mêmes mesurements, tags et fields sont appelées des séries temporelles (*time series*)

Les données sont organisées dans des *buckets*. Chaque bucket peut contenir plusieurs mesures (*mesurements*)  différentes.
Une mesure contiennent un timestamp et plusieurs *tags* et *fields*.
Les tags sont des paires clef-valeur dont les valeurs ne changent pas souvent. Ils sont indexés donc ils permettents d'organiser rapidement les données. Par exemple, cela peut être le lieu d'une expérience fixe, ou le nom de l'appareillage.
Les fields sont des paires clef-valeurs qui changent au cours du temps. Ils ne sont pas indexées. Par exemple, cela peut être la température ou la pression mesurée.



![alt text](data/image.png)

*Source: https://docs.influxdata.com*

In [3]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

In [4]:
# Paramètres de connexion
url = "http://influxdb:8086"
token = "my-secret-token"
org = "my-org"
bucket = "my-bucket"

In [5]:
# Création du client
client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

print("✅ Connexion réussie à InfluxDB!")

✅ Connexion réussie à InfluxDB!


In [6]:
# Population de la base de données
# Chemin du fichier contenant les données
FILENAME = "data/air-sensor-data.txt"
print("Population de la base de données avec les données du fichier...", FILENAME)
with open(FILENAME, "r") as file:
    for line in file:
        write_api.write(bucket=bucket, org=org, record=line)
print("Population terminée!")


Population de la base de données avec les données du fichier... data/air-sensor-data.txt
Population terminée!


### 📊 Insertion de Données
Nous allons insérer une mesure simple (ex: température d'un capteur).


In [23]:
data = "temperature,sensor_id=1 value=22.5"
write_api.write(bucket=bucket, org=org, record=data)
print("✅ Donnée insérée avec succès!")

✅ Donnée insérée avec succès!


### 🔍 Lecture des Données
Nous allons récupérer les données insérées à l'aide d'une requête.


In [25]:
query = f"""
from(bucket: "{bucket}")
|> range(start: -1h)
"""

result = query_api.query(org=org, query=query)

# Affichage des résultats
for table in result:
  for record in table.records:
      print(f"📌 {record.values}")

print("✅ Requête exécutée avec succès!")


📌 {'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 2, 18, 13, 5, 41, 702135, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2025, 2, 18, 14, 5, 41, 702135, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2025, 2, 18, 13, 23, 42, 465555, tzinfo=datetime.timezone.utc), '_value': 22.5, '_field': 'value', '_measurement': 'temperature', 'sensor_id': '1'}
📌 {'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 2, 18, 13, 5, 41, 702135, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2025, 2, 18, 14, 5, 41, 702135, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2025, 2, 18, 13, 33, 33, 35821, tzinfo=datetime.timezone.utc), '_value': 22.5, '_field': 'value', '_measurement': 'temperature', 'sensor_id': '1'}
📌 {'result': '_result', 'table': 0, '_start': datetime.datetime(2025, 2, 18, 13, 5, 41, 702135, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2025, 2, 18, 14, 5, 41, 702135, tzinfo=datetime.timezone.utc), 

### ❌ Fermeture de la connexion
Toujours fermer la connexion après utilisation pour libérer les ressources.


In [ ]:
client.close()
print("✅ Connexion fermée.")

✅ Connexion fermée.
